In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import sklearn.metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss, jaccard_score
from sklearn.metrics import f1_score
from kmodes.kmodes import KModes
from sklearn.model_selection import GridSearchCV
from global_variables import sexo_options
from global_variables import estado_civil_options
from global_variables import transporte_options
from global_variables import modalidad_options
from global_variables import conducta_options
from global_variables import comuna_options
from global_variables import lugar_options
from global_variables import bien_options
from global_variables import grupo_edad

In [2]:
hurto_tp=pd.read_csv("../datasets_clean/hurto_tpc.csv", delimiter=";", encoding="utf-8")
hurto_ml = hurto_tp 
df1 = hurto_tp 

In [3]:
df1 = df1.drop(['Fecha','Sexo','Edad','Año'], axis=1)

In [4]:
km = KModes(n_clusters=3, init="Huang", n_init=10,verbose=1)
df1.fillna("Desconocido", inplace = True)
df1 = df1.astype(str)
clusters= km.fit_predict(df1)

print("Centroides de los clusters")
print(km.cluster_centroids_)
print("Clusters asignados a cada punto de datos:")
print(clusters)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 4976, cost: 71884.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 9230, cost: 65827.0
Run 2, iteration: 2/100, moves: 1085, cost: 65827.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 10757, cost: 66282.0
Run 3, iteration: 2/100, moves: 1861, cost: 66282.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 8753, cost: 67316.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 7660, cost: 65827.0
Run 5, iteration: 2/100, moves: 4, cost: 65827.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/100, moves: 4738, cost: 71423.0
Run 6, iteration: 2/100, moves: 174, cost: 71423.0
Ini

In [5]:
def encontrar_posicion(valor, lista):
    for i, elemento in enumerate(lista):
        if elemento == valor:
            return i
    return -1  # Valor no encontrado

In [6]:
for i in range(len(hurto_ml)):
    hurto_ml.loc[i,"Sexo"]=encontrar_posicion(hurto_ml.loc[i,"Sexo"],sexo_options)
    hurto_ml.loc[i,"Estado_civil"]=encontrar_posicion(hurto_ml.loc[i,"Estado_civil"],estado_civil_options)
    hurto_ml.loc[i,"Comuna"]=encontrar_posicion(hurto_ml.loc[i,"Comuna"],comuna_options)
    hurto_ml.loc[i,"Transporte"]=encontrar_posicion(hurto_ml.loc[i,"Transporte"],transporte_options)
    hurto_ml.loc[i,"Modalidad"]=encontrar_posicion(hurto_ml.loc[i,"Modalidad"],modalidad_options)
    hurto_ml.loc[i,"Conducta"]=encontrar_posicion(hurto_ml.loc[i,"Conducta"],conducta_options)
    hurto_ml.loc[i,"Lugar"]=encontrar_posicion(hurto_ml.loc[i,"Lugar"],lugar_options)
    hurto_ml.loc[i,"Bien"]=encontrar_posicion(hurto_ml.loc[i,"Bien"],bien_options)

In [7]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X = hurto_ml.drop(['Fecha','Sexo'], axis=1)  # Elimina las columnas
y = hurto_ml['Sexo']  # Define tu objetivo, en este caso 'Sexo'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
def compare_metrics(learners,X,y,metrics,train_size,repeats):
    strlearners=[]
    for i in range(0,len(learners)):
        strlearners.append(str(learners[i]))  
    df=pd.DataFrame(columns=metrics,index=strlearners)
    for i in range (0,len(learners)):
        for j in range(0,len(metrics)):
            scores=[]
            for k in range(repeats):
                X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=train_size)
                learners[i].fit(X_train,y_train)
                y_hat = learners[i].predict(X_test)
                y_prob = learners[i].predict_proba(X_test)
                if metrics[j]=="Log loss":
                    scores.append(sklearn.metrics.log_loss(y_test, y_prob))
                if metrics[j]=="Brier Score Loss":
                    scores.append(sklearn.metrics.brier_score_loss(y_test, y_prob[:,1]))
                if metrics[j]=="Accuracy":
                    scores.append(sklearn.metrics.accuracy_score(y_test, y_hat))
                if metrics[j]=="Precision":
                    scores.append(sklearn.metrics.precision_score(y_test, y_hat))
                if metrics[j]=="F1":
                    scores.append(sklearn.metrics.f1_score(y_test, y_hat))
                if metrics[j]=="Recall":
                    scores.append(sklearn.metrics.recall_score(y_test, y_hat))
                if metrics[j]=="Auroc":
                    scores.append(sklearn.metrics.roc_auc_score(y_test, y_prob[:,1]))
            df.iloc[i,j]=np.mean(scores)
    return df

In [9]:
metricas=["Log loss","Brier Score Loss","Accuracy","Precision","F1","Recall","Auroc"]
decision_tree=DecisionTreeClassifier()
naive_bayes=GaussianNB()
kNN=KNeighborsClassifier()
log_regression = LogisticRegression()
RandomForest = RandomForestClassifier()
elearn=[decision_tree,naive_bayes,kNN,log_regression,RandomForest]

In [10]:
#hurto_ml_scores  = compare_metrics(elearn,X,y,metricas,0.8,20)
#hurto_ml_scores

##### Multietiqueta

In [11]:
bins = [-1, 12, 18, 30, 60, float('inf')]
labels = ['Niño', 'Adolescente', 'Adulto joven', 'Adulto', 'Adulto mayor']

# Utiliza pd.cut para crear los intervalos y asignar las etiquetas
hurto_ml['grupo_edad'] = pd.cut(hurto_ml['Edad'], bins=bins, labels=labels, right=False)

#Transformación a texto
hurto_ml['grupo_edad'] = hurto_ml['grupo_edad'].astype(str)

In [12]:
for i in range(len(hurto_ml)):
    hurto_ml.loc[i,"grupo_edad"]=encontrar_posicion(hurto_ml.loc[i,"grupo_edad"],grupo_edad)

In [13]:
# Dividir el conjunto de datos en características (X) y etiquetas (y)
X = hurto_ml.drop(['Fecha', 'grupo_edad', 'Sexo','Edad','Año'], axis=1)
y = list(zip(hurto_ml['grupo_edad'],hurto_ml['Sexo']))

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un clasificador multi-etiqueta (por ejemplo, Random Forest)
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

# Entrenar el modelo
classifier.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = classifier.predict(X_test)

# Evaluar el rendimiento del modelo
print(classifier.score(X_train, np.array(y_train)))

0.6911650107149815


Definición de función para la identificación de hiperparámetros óptimos mediante grid_search

In [14]:
# param_grid = {
#     'estimator__n_estimators': [100, 200, 300],
#     'estimator__max_depth': [None, 10, 20, 30]
# }

# grid_search = GridSearchCV(
#     estimator=classifier,
#     param_grid=param_grid,
#     scoring='accuracy',  # Cambia la métrica según tus necesidades
#     cv=5
# )

Ejecución 

In [15]:
#grid_search.fit(X_train, y_train)

In [16]:
# best_params = grid_search.best_params_
# best_params

In [17]:
# best_estimator = grid_search.best_estimator_
# best_estimator

Generación de Input para clasificación

In [22]:
test = [(2,2,2,3,7,2,0)]

Predicción

In [24]:
y_pred = classifier.predict(test)

c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\Asus\miniconda3\envs\project\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [25]:
y_pred

array([[2, 1]])

In [26]:
print('Género predicción:', sexo_options[y_pred[0, 1]],'\n','Grupo edad predicción:', grupo_edad[y_pred[0, 0]])

Género predicción: Mujer 
 Grupo edad predicción: Adulto joven


In [29]:
sexo_options[0]

'Hombre'

Cargue del modelo en formato PKL

In [20]:
import pickle

with open("../api_generation/model.pkl","wb") as archivo:
    pickle.dump(classifier,archivo)

In [27]:
diccio={0:0,1:2}

In [28]:
{"prediction":[0:[0:3
                  1:0]]}

dict